In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
symbol_list = ['BTC', 'EOS', 'ETH', 'XRP', 'LTC']

date_list = [
    'start=2019-07-01T00%3A00%3A00.000Z&end=2020-01-01T00%3A00%3A00.000Z',
    'start=2019-01-01T00%3A00%3A00.000Z&end=2019-07-01T00%3A00%3A00.000Z',
    'start=2018-07-01T00%3A00%3A00.000Z&end=2019-01-01T00%3A00%3A00.000Z',
    'start=2018-01-01T00%3A00%3A00.000Z&end=2018-07-01T00%3A00%3A00.000Z',
    'start=2017-07-01T00%3A00%3A00.000Z&end=2018-01-01T00%3A00%3A00.000Z',
    'start=2017-01-01T00%3A00%3A00.000Z&end=2017-07-01T00%3A00%3A00.000Z',
]

for symbol in symbol_list[:]:
    print(symbol)
    df = pd.DataFrame()
    for date in date_list:
        print(date)
        _df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/%s-USDT/candles?granularity=86400&%s' % (symbol, date))
        if len(df) == 0:
            df = _df
        else:
            df = df.append(_df, ignore_index=True)
        time.sleep(1)
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
    df['date'] = df['date'].apply(pd.to_datetime)
    df['date'] = df['date'] + pd.Timedelta(hours=8)
    df.head(2)
    df.tail(2)

    datafile = '../database/market/%s_OKEX.csv' % symbol
    df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

In [ ]:
import time
import urllib

symbol_list = ['BTC', 'EOS', 'ETH', 'XRP', 'LTC']

now = time.strftime('%Y-%m-%d-%H:00:00')
# now = '2019-03-28-00:00:00'
now = pd.to_datetime(now)
now = now - pd.Timedelta(hours=8)

for symbol in symbol_list[:]:
    print(symbol)
    df = pd.DataFrame()
    end_time = now
    start_time = end_time - pd.Timedelta(days=7)
    
    while end_time.strftime('%Y-%m-%d') > '2017-11-01':
        start_time, end_time
        end = end_time.strftime('%Y-%m-%dT%H:00:00.000Z')
        end = urllib.parse.quote(end)
        start = start_time.strftime('%Y-%m-%dT%H:00:00.000Z')
        start = urllib.parse.quote(start)
        url = 'https://www.okex.me/api/spot/v3/instruments/%s-USDT/candles?granularity=3600&start=%s&end=%s' % (symbol, start, end)
        _df = pd.read_json(url)
        if len(_df) == 0:
            break
        if len(df) == 0:
            df = _df
        else:
            df = df.append(_df, ignore_index=True)
        end_time = start_time
        start_time = start_time - pd.Timedelta(days=7)
        time.sleep(1)
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
    df['date'] = df['date'].apply(pd.to_datetime)
    df['date'] = df['date'] + pd.Timedelta(hours=8)
    df.head(2)
    df.tail(2)

    datafile = '../database/market/%s_OKEX_HOURS.csv' % symbol
    df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')